In [1]:
import os
import math
import cv2
import dlib
import multiprocessing

# Recortar rosto do animal e salvar a imagem

In [2]:
def process_image(img_path, input_dir, output_dir):
    try:
        detector = dlib.cnn_face_detection_model_v1('dlib_models/dogHeadDetector.dat')
        filename, ext = os.path.splitext(os.path.basename(img_path))
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        dets = detector(img, upsample_num_times=1)
        
        for i, d in enumerate(dets):

            x1, y1 = d.rect.left(), d.rect.top()
            x2, y2 = d.rect.right(), d.rect.bottom()
            
            roi = img[y1:y2, x1:x2]
            
            output_subdir = os.path.join(output_dir, os.path.relpath(os.path.dirname(img_path), input_dir))
            os.makedirs(output_subdir, exist_ok=True)
            output_filename = f"{filename}_face{i+1}.jpg"
            output_path = os.path.join(output_subdir, output_filename)
            cv2.imwrite(output_path, cv2.cvtColor(roi, cv2.COLOR_RGB2BGR))
    except Exception as e:
        print(f"Erro na imagem {img_path}: {str(e)}")

def process_images_in_directory(input_dir, output_dir):
    with multiprocessing.Pool() as pool:
        image_paths = []
        for root, dirs, files in os.walk(input_dir):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                    img_path = os.path.join(root, file)
                    image_paths.append(img_path)

        pool.starmap(process_image, [(img_path, input_dir, output_dir) for img_path in image_paths])

In [3]:
# Exemplo de uso:
input_directory = "images"
output_directory = "images_processed"
process_images_in_directory(input_directory, output_directory)

## Train e Test

In [4]:
import os
import shutil
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from sklearn.model_selection import train_test_split
from shutil import copyfile

2023-09-25 21:56:41.839460: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-25 21:56:41.841182: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 21:56:41.883345: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-25 21:56:41.883860: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-25 21:56:42.567318: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [5]:
src_dir = 'images_processed'
train_dir = 'images_processed/train'
test_dir = 'images_processed/test'

In [6]:
def clean_old_test_train(path):
    if os.path.exists(path):
        shutil.rmtree(path)  # Use rmtree para remover um diretório e seu conteúdo
        print(f"Diretório '{path}' removido com sucesso.")
    else:
        print(f"Diretório '{path}' não existe.")

In [7]:
clean_old_test_train(train_dir)
clean_old_test_train(test_dir)

Diretório 'images_processed/train' removido com sucesso.
Diretório 'images_processed/test' removido com sucesso.


In [8]:
datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [9]:
def generate_and_save_images(file):
    src_file = os.path.join(src_label_dir, file)
    dst_file = os.path.join(train_label_dir, file)
    
    if not os.path.isfile(src_file):
        return
    
    img = load_img(src_file)  # this is a PIL image
    img = img.resize((250, 250))  # Resize the image
    img.save(dst_file)  # Save the resized image
    
    x = img_to_array(img)  # this is a Numpy array with shape (1, 250, 250)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 1, 250, 250)

    i = 0
    for batch in datagen.flow(x, batch_size=1,
                              save_to_dir=train_label_dir, save_prefix='aug', save_format='jpeg'):
        i += 1
        if i > 10:
            break  # otherwise the generator would loop indefinitely

In [10]:
labels = os.listdir(src_dir)  #Obteras labels a partir dos subdiretórios

for label in labels:
    src_label_dir = os.path.join(src_dir, label)
    train_label_dir = os.path.join(train_dir, label)
    test_label_dir = os.path.join(test_dir, label)
    
    os.makedirs(train_label_dir, exist_ok=True)
    os.makedirs(test_label_dir, exist_ok=True)
    
    files = os.listdir(src_label_dir)
    train_files, test_files = train_test_split(files, test_size=0.2, random_state=42)
    
    with multiprocessing.Pool() as p:
        p.map(generate_and_save_images, train_files)
    
    for file in test_files:
        src_file = os.path.join(src_label_dir, file)
        dst_file = os.path.join(test_label_dir, file)
        copyfile(src_file, dst_file)

## Data augmentation

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255)

In [12]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(250, 250),
                                                    batch_size=32,
                                                    class_mode='categorical')

Found 10923 images belonging to 4 classes.


In [13]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                  target_size=(250, 250),
                                                  batch_size=32,
                                                  class_mode='categorical')

Found 1215 images belonging to 4 classes.


## DenseNet - 121

In [15]:
from tensorflow.keras.applications import DenseNet121
from keras.callbacks import EarlyStopping
import tensorflow as tf

In [16]:
def create_and_train_model_densenet_121(train_generator, test_generator, num_epochs=30, initial_learning_rate=0.001, final_learning_rate=1e-5):
    
    #Callback EarlyStopping
    early_stopping = EarlyStopping(monitor='val_accuracy',
                                   patience=15,
                                   verbose=1,
                                   restore_best_weights=True)
    
    densenet_base = tf.keras.applications.DenseNet121(weights='imagenet',
                                                      include_top=False,
                                                      input_shape=(250, 250, 3))
    
    #Congelar as camadas do DenseNet121
    densenet_base.trainable = False
    
    #Geração do valor de steps
    total_training_examples = int(train_generator.samples)
    batch_size = train_generator.batch_size
    steps_per_epoch = math.ceil(total_training_examples / batch_size)
    validation_steps = test_generator.samples // test_generator.batch_size
    

    model_densenet = tf.keras.models.Sequential([
        densenet_base,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    
    #Taxa de decaimento
    decay_rate = (final_learning_rate / initial_learning_rate) ** (1 / num_epochs)
    
    #Função de agendamento de taxa de aprendizado
    def lr_schedule(epoch):
        current_learning_rate = initial_learning_rate * (decay_rate ** epoch)
        return current_learning_rate
    
    model_densenet.compile(loss='categorical_crossentropy',
                           optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
                           metrics=['accuracy'])
    
    #Função lr_schedule para agendar a taxa de aprendizado
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

    history_densenet = model_densenet.fit(train_generator,
                                          steps_per_epoch=steps_per_epoch,
                                          epochs=num_epochs,
                                          validation_data=test_generator,
                                          validation_steps=validation_steps,
                                          callbacks=[early_stopping, lr_callback])
    
    return model_densenet, history_densenet

In [ ]:
model_densenet_121, history_densenet_121 = create_and_train_model_densenet_121(train_generator, test_generator, num_epochs=100)

2023-09-25 22:00:05.558085: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-25 22:00:05.558752: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/100
342/342 [==============================] - 471s 1s/step - loss: 3.2547 - accuracy: 0.5500 - val_loss: 0.8572 - val_accuracy: 0.6410 - lr: 0.0010
Epoch 2/100
342/342 [==============================] - 464s 1s/step - loss: 0.8831 - accuracy: 0.6041 - val_loss: 0.8817 - val_accuracy: 0.6529 - lr: 9.5499e-04
Epoch 3/100
342/342 [==============================] - 471s 1s/step - loss: 0.7952 - accuracy: 0.6375 - val_loss: 0.8430 - val_accuracy: 0.6436 - lr: 9.1201e-04
Epoch 4/100
342/342 [==============================] - 460s 1s/step - loss: 0.7309 - accuracy: 0.6683 - val_loss: 0.8424 - val_accuracy: 0.6529 - lr: 8.7096e-04
Epoch 5/100
342/342 [==============================] - 460s 1s/step - loss: 0.6803 - accuracy: 0.6896 - val_loss: 0.8440 - val_accuracy: 0.6427 - lr: 8.3176e-04
Epoch 6/100
342/342 [==============================] - 521s 2s/step - loss: 0.6472 - accuracy: 0.6961 - val_loss: 0.8837 - val_accuracy: 0.6613 - lr: 7.9433e-04
Epoch 7/100
342/342 [=================

In [ ]:
test_generator.samples

In [ ]:
test_loss_densenet_121, test_acc_densenet_121 = model_densenet_121.evaluate(test_generator, verbose=2)
print('\nTest accuracy:', test_acc_densenet_121)

## DenseNet 201 - Com decaimento de LR

In [ ]:
from tensorflow.keras.applications import DenseNet201
from keras.callbacks import EarlyStopping
import tensorflow as tf

In [ ]:
def create_and_train_model_densenet_201(train_generator, test_generator, num_epochs=30, initial_learning_rate=0.001, final_learning_rate=1e-5):
    
    #Callback EarlyStopping
    early_stopping = EarlyStopping(monitor='val_accuracy',
                                  patience=15,
                                  verbose=1,
                                  restore_best_weights=True)

    
    densenet_base = tf.keras.applications.DenseNet201(weights='imagenet',
                                                      include_top=False,
                                                      input_shape=(250, 250, 3))
    
    #Congelar as camadas do DenseNet201
    densenet_base.trainable = False

    #Geração do valor de steps
    total_training_examples = train_generator.samples
    batch_size = train_generator.batch_size
    steps_per_epoch = total_training_examples / batch_size
    validation_steps = test_generator.samples // test_generator.batch_size

    model_densenet = tf.keras.models.Sequential([
        densenet_base,
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    
    #Taxa de decaimento da LR
    decay_rate = (final_learning_rate / initial_learning_rate) ** (1 / num_epochs)
    
    #Função de agendamento de taxa de aprendizado
    def lr_schedule(epoch):
        current_learning_rate = initial_learning_rate * (decay_rate ** epoch)
        return current_learning_rate

    model_densenet.compile(loss='categorical_crossentropy',
                           optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),  # Use learning_rate
                           metrics=['accuracy'])
    
    #Função lr_schedule para agendar a taxa de aprendizado
    lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

    history_densenet = model_densenet.fit(train_generator,
                                          steps_per_epoch=steps_per_epoch,
                                          epochs=num_epochs,
                                          validation_data=test_generator,
                                          validation_steps=validation_steps,
                                          callbacks=[early_stopping, lr_callback])
    
    return model_densenet, history_densenet

In [ ]:
model_densenet_201, history_densenet_201 = create_and_train_model_densenet_201(train_generator, test_generator, num_epochs=100)

In [ ]:
test_loss_densenet_201, test_acc_densenet_201 = model_densenet_201.evaluate(test_generator, verbose=2)
print('\nTest accuracy:', test_acc_densenet_201)

## Salvar modelos

In [ ]:
def save_model(model, filename):
    model.save('cnn_models/'+filename)

In [ ]:
save_model(model_densenet_121, 'model_densenet_121.h5')
save_model(model_densenet_201, 'model_densenet_201.h5')

## Predizer imagens

In [ ]:
def process_image_predict(img_path, target_size=(250, 250)):
    try:
        detector = dlib.cnn_face_detection_model_v1('dlib_models/dogHeadDetector.dat')
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        dets = detector(img, upsample_num_times=1)

        # Se nenhuma detecção de face for encontrada, retorne uma imagem em branco
        if len(dets) == 0:
            print("Não foram encontrados rostos na imagem")
            return np.zeros((target_size[0], target_size[1], 3), dtype=np.uint8)

        # Desenhe retângulos nas detecções
        for i, d in enumerate(dets):
            x1, y1 = d.rect.left(), d.rect.top()
            x2, y2 = d.rect.right(), d.rect.bottom()
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        # Converta a imagem resultante de volta para BGR para salvar ou exibir
        img_result = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        # Redimensione a imagem resultante para o tamanho desejado
        img_result = cv2.resize(img_result, target_size)

        return img_result

    except Exception as e:
        print(f"Erro na imagem {img_path}: {str(e)}")

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np

def image_class_predict(model, image_path):
    classes = ["angry", "happy", "sad"]
    # Carregue e pré-processe a imagem de entrada
    img = process_image_predict(image_path)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0  # Normalização (se necessário)

    # Faça a previsão
    predictions = model.predict(img_array)

    # Obtenha a classe prevista
    class_index = np.argmax(predictions)

    return class_index, classes[class_index]

## Resultado Densenet 121

In [ ]:
imagem = 'example.jpeg'
classe_prevista_index, classe_prevista = image_class_predict(model_densenet_121, imagem)

print(f'A imagem é da classe: {classe_prevista}')

## Resultado Densenet 201

In [ ]:
imagem = 'example.jpeg'
classe_prevista_index, classe_prevista = image_class_predict(model_densenet_201, imagem)

print(f'A imagem é da classe: {classe_prevista}')